In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル①】　準備

#@markdown ■■■■■■■■■■■■■■■■■■

# qt5

! sudo apt-get install python3-pyqt5  
! sudo apt-get install pyqt5-dev-tools
! sudo apt-get install qttools5-dev-tools


print("■■■■■■■■■■■■■■■■■■")
print("■　準備が完了しました。")
print("■■■■■■■■■■■■■■■■■■")

In [0]:
from math import atan2, acos, asin, cos, sin, degrees, isnan, isclose, sqrt, pi, isinf, radians
from PyQt5.QtGui import QQuaternion, QVector3D, QVector2D, QMatrix4x4, QVector4D
import traceback

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル②】　計算

#@markdown ■■■■■■■■■■■■■■■■■■

# 指定された3点と半径を通る球の中心点を求める
# https://oshiete.goo.ne.jp/qa/195295.html
# https://okwave.jp/qa/q9467739.html
def calc_sphere_center(pv, wv, nv, r):
    x1 = pv.x()
    y1 = pv.y()
    z1 = pv.z()
    x2 = wv.x()
    y2 = wv.y()
    z2 = wv.z()
    x3 = nv.x()
    y3 = nv.y()
    z3 = nv.z()

    m = (pv + wv + nv) / 3

    try:
        tm01=x1**2-x2**2+y1**2-y2**2+z1**2-z2**2
        tm02=x1**2-x3**2+y1**2-y3**2+z1**2-z3**2
        tm11=-2*(x1-x2)*(z1-z3)+2*(x1-x3)*(z1-z2)
        tm12=-2*(y1-y2)*(z1-z3)+2*(y1-y3)*(z1-z2)
        tm13=tm01*(z1-z3)-tm02*(z1-z2)
        tm21=-2*(x1-x2)*(y1-y3)+2*(x1-x3)*(y1-y2)
        tm22=-2*(z1-z2)*(y1-y3)+2*(z1-z3)*(y1-y2)
        tm23=tm01*(y1-y3)-tm02*(y1-y2)
        tma=1+tm11**2/tm12**2+tm21**2/tm22**2
        tmb=-2*x1+2*(y1+tm13/tm12)*tm11/tm12+2*(z1+tm23/tm22)*tm21/tm22
        tmc=x1**2+(y1+tm13/tm12)**2+(z1+tm23/tm22)**2-r**2
        xq1=(-tmb+sqrt(abs(tmb**2-4*tma*tmc)))/2/tma
        xq2=(-tmb-sqrt(abs(tmb**2-4*tma*tmc)))/2/tma
        yq1=-tm13/tm12-tm11/tm12*xq1
        yq2=-tm13/tm12-tm11/tm12*xq2
        zq1=-tm23/tm22-tm21/tm22*xq1
        zq2=-tm23/tm22-tm21/tm22*xq2

        c1 = QVector3D(xq1, yq1, zq1)
        c2 = QVector3D(xq2, yq2, zq2)

        if c1 == c2:
            # 重解
            return c1, r

        if c1.distanceToPoint(m) < c2.distanceToPoint(m):
            # 3点の中間に近い方を返す
            return c1, r

        return c2, r
    except ZeroDivisionError as e:
        # print("ゼロ割エラー1")
        # print(traceback.format_exc())

        if round(x1,1) == round(x2,1) == round(x3,1):
            # 同じ値の場合、2次元円として求める
            cx, cy, r = calc_circle_center(y1, z1, y2, z2, y3, z3)
            return QVector3D(x1, cx, cy), r
        
        if round(y1,1) == round(y2,1) == round(y3,1):
            cx, cy, r = calc_circle_center(x1, z1, x2, z2, x3, z3)
            return QVector3D(cx, y1, cy), r
        
        if round(z1,1) == round(z2,1) == round(z3,1):
            cx, cy, r = calc_circle_center(x1, y1, x2, y2, x3, y3)
            return QVector3D(cx, cy, z1), r
    
    return QVector3D(), r


# http://www.iot-kyoto.com/satoh/2016/01/29/tangent-003/
# http://nobutina.blog86.fc2.com/blog-entry-674.html
def calc_circle_center(x1, y1, x2, y2, x3, y3):

    G=( y2*x1-y1*x2 +y3*x2-y2*x3 +y1*x3-y3*x1 )

    try:
        Xc= ((x1*x1+y1*y1)*(y2-y3)+(x2*x2+y2*y2)*(y3-y1)+(x3*x3+y3*y3)*(y1-y2))/(2*G)
        Yc=-((x1*x1+y1*y1)*(x2-x3)+(x2*x2+y2*y2)*(x3-x1)+(x3*x3+y3*y3)*(x1-x2))/(2*G)

        Xd=(((x1*x1+y1*y1)-(x2*x2+y2*y2))*(y2-y3)-((x2*x2+y2*y2)-(x3*x3+y3*y3))*(y1-y2))/(2*((x1-x2)*(y2-y3)-(x2-x3)*(y1-y2)))
        Yd=(((y1*y1+x1*x1)-(y2*y2+x2*x2))*(x2-x3)-((y2*y2+x2*x2)-(y3*y3+x3*x3))*(x1-x2))/(2*((y1-y2)*(x2-x3)-(y2-y3)*(x1-x2)))

        G=2 * sqrt( (x1 - Xc) * (x1 - Xc) + (y1 - Yc) * (y1 - Yc) )

        return Xd, Yd, G/2
    except ZeroDivisionError as e:
        # print("ゼロ割エラー2")
        # print(traceback.format_exc())

        if round(x1,1) == round(x2,1) == round(x3,1):
            G = sqrt((y1 + y2 + y3) ** 2)
            return (x1 + x2 + x3) / 3, (y1 + y2 + y3) / 3, G

        G = sqrt((x1 + x2 + x3) ** 2)
        return (x1 + x2 + x3) / 3, (y1 + y2 + y3) / 3, G

    return 0, 0, 0



#@markdown 三点の位置のXYZをそれぞれの欄に入力して、セルを実行してください。
#@markdown 
#@markdown 回転支点の位置とその軸制限が求められます。
#@markdown 
#@markdown 三点の位置は、刀を真上から見た時の中心線を通る頂点を選んでください
#@markdown 
#@markdown 大体の場合、XYZのどれかがとても大きな値（打刀で30以上くらい）になります。
#@markdown 
#@markdown ---

#@markdown ### 刀の刃区（根元）根元のXYZ

point1_x = 3.321042  #@param {type: "number"}
point1_y = 5.73096  #@param {type: "number"}
point1_z = -3.896624  #@param {type: "number"}

#@markdown ---

#@markdown ### 刃区と切っ先の中間くらいのXYZ

point2_x = -0.4420419  #@param {type: "number"}
point2_y = 5.700311  #@param {type: "number"}
point2_z = -3.902588  #@param {type: "number"}

#@markdown ---

#@markdown ### 刀の切っ先（先端）のXYZ

point3_x = -4.205778  #@param {type: "number"}
point3_y = 5.968236  #@param {type: "number"}
point3_z = -3.908525  #@param {type: "number"}

point1_vec = QVector3D(point1_x, point1_y, point1_z)
point2_vec = QVector3D(point2_x, point2_y, point2_z)
point3_vec = QVector3D(point3_x, point3_y, point3_z)

print("point1 %s" % (point1_vec))
print("point2 %s" % (point2_vec))
print("point3 %s" % (point3_vec))

# 半径は3点間の距離の最長の半分
r = max(point1_vec.distanceToPoint(point2_vec), point1_vec.distanceToPoint(point3_vec),  point2_vec.distanceToPoint(point3_vec)) / 2

# 半径と原点を求める
center, radius = calc_sphere_center(point1_vec, point2_vec, point3_vec, r)

# 鞘回転支点の軸制限
fixed_axis1 = QVector3D.crossProduct(center, (point2_vec - point1_vec).normalized())

# 刀回転支点の軸制限
fixed_axis2 = QVector3D.crossProduct(center, (point2_vec - point3_vec).normalized())

print("■■■■■■■■■■■■■■■■■■")
print("■　回転支点の位置")
print("■　x: %s" % (center.x()))
print("■　y: %s" % (center.y()))
print("■　z: %s" % (center.z()))
# print("■　--------")
# print("■　鞘回転支点の軸制限")
# print("■　x: %s" % (fixed_axis1.x()))
# print("■　y: %s" % (fixed_axis1.y()))
# print("■　z: %s" % (fixed_axis1.z()))
# print("■　--------")
# print("■　刀回転支点の軸制限")
# print("■　x: %s" % (fixed_axis2.x()))
# print("■　y: %s" % (fixed_axis2.y()))
# print("■　z: %s" % (fixed_axis2.z()))
print("■■■■■■■■■■■■■■■■■■")
